In [6]:
import osmnx as ox
import geopandas as gpd
import folium
import io
from PIL import Image
import matplotlib.pyplot as plt

# 1. COLETA DE DADOS (Seu código, com pequenas melhorias)

# Lista dos nomes dos seus equipamentos públicos
nomes_equipamentos = [
    "Biblioteca Mário de Andrade, São Paulo, Brasil",
    "Campo de Marte, São Paulo, Brasil",
    "Centro Cultural da Penha, São Paulo, Brasil",
    "Centro Cultural Tendal da Lapa, São Paulo, Brasil",
    "Centro Cultural Vila Itororó, São Paulo, Brasil",
    "Centro TEA, São Paulo, Brasil",
    "CEU Arthur Alvim - Abdias do Nascimento, São Paulo, Brasil",
    "CEU Carrão, São Paulo, Brasil",
    "CEU Cidade Tiradentes, São Paulo, Brasil",
    "CEU Freguesia do Ó - Esperança Garcia, São Paulo, Brasil",
    "CEU José Bonifácio, São Paulo, Brasil",
    "CEU Parque do Carmo, São Paulo, Brasil",
    "CEU Parque Novo Mundo, São Paulo, Brasil",
    "CEU Luiz Gama (Pinheirinho D'agua), São Paulo, Brasil",
    "CEU São Miguel - Luiz Melodia, São Paulo, Brasil",
    "CEU Taipas - Profª Maria Beatriz Nascimento, São Paulo, Brasil",
    "CEU Tremembé, São Paulo, Brasil",
    "CEU Vila Alpina, São Paulo, Brasil",
    "PARADA 14 BIS, São Paulo, Brasil",
    "Terminal Antônio Estevão de Carvalho, São Paulo, Brasil",
    "Term. Aricanduva, São Paulo, Brasil",
    "Terminal Cidade Tiradentes, São Paulo, Brasil",
    "Terminal Hidroviário Parque Linear Cantinho do Céu, São Paulo, Brasil",
    "Terminal Hidroviario Mar Paulista, São Paulo, Brasil",
    "Terminal Metrô Itaquera, São Paulo, Brasil",
    "Terminal Mercado, São Paulo, Brasil",
    "Estação Clínicas, São Paulo, Brasil",
    "Eldorado B/C, São Paulo, Brasil",
    "Terminal Parque Dom Pedro II, São Paulo, Brasil",
    "Terminal Penha, São Paulo, Brasil",
    "Terminal Sacomã, São Paulo, Brasil",
    "Term. São Miguel, São Paulo, Brasil",
    "Terminal Sapopemba, São Paulo, Brasil",
    "Terminal Carrão, São Paulo, Brasil",
    "Terminal Vila Prudente, São Paulo, Brasil",
]

# Defina as tags do OpenStreetMap para buscar por edificações (e outras feições)
# Ampliamos a busca para além de 'building' para encontrar terminais, etc.
tags = {"building": True, "amenity": True, "public_transport": True}

lista_de_gdfs = []
for nome in nomes_equipamentos:
    try:
        # A função 'features_from_place' do OSMnx faz a busca
        gdf = ox.features_from_place(nome, tags)
        # Adiciona o nome da busca como uma coluna para referência futura
        gdf['nome_busca'] = nome
        lista_de_gdfs.append(gdf)
        print(f"Feição para '{nome}' encontrada com sucesso!")
    except Exception as e:
        print(f"Não foi possível encontrar uma feição para '{nome}': {e}")

# Concatene todos os GeoDataFrames em um só
if not lista_de_gdfs:
    print("\nNenhuma feição foi encontrada. O mapa não pode ser gerado.")
else:
    todos_os_poligonos = gpd.pd.concat(lista_de_gdfs, ignore_index=True)

    # Remove geometrias que não sejam polígonos/multipolígonos para evitar erros
    todos_os_poligonos = todos_os_poligonos[todos_os_poligonos.geometry.type.isin(['Polygon', 'MultiPolygon'])]

    # 2. CRIAÇÃO DO MAPA COM FOLIUM

    # Calcula o ponto central do mapa a partir da média das coordenadas
    lat_media = todos_os_poligonos.unary_union.centroid.y
    lon_media = todos_os_poligonos.unary_union.centroid.x

    # Inicia o mapa do Folium
    m = folium.Map(location=[lat_media, lon_media], zoom_start=11, tiles="cartodbpositron")

    # Adiciona os polígonos ao mapa
    for _, r in todos_os_poligonos.iterrows():
        # Simplifica a geometria para evitar problemas de performance no Folium
        sim_geo = gpd.GeoSeries(r.geometry).simplify(tolerance=0.0001)
        geo_j = sim_geo.to_json()
        
        # Define o nome que aparecerá no pop-up (label)
        # Prioriza a tag 'name' do OSM, se não existir, usa o nome da busca
        nome_label = r.get('name', r['nome_busca'])

        # Adiciona a geometria ao mapa
        geo_json_obj = folium.GeoJson(
            data=geo_j,
            style_function=lambda x: {'color': 'blue', 'fillColor': 'lightblue', 'weight': 2, 'fillOpacity': 0.6}
        )
        
        # Adiciona um pop-up com o nome ao clicar
        folium.Popup(f"<b>{nome_label}</b><br><small>Busca: {r['nome_busca']}</small>").add_to(geo_json_obj)
        
        # Adiciona a camada de GeoJson ao mapa
        geo_json_obj.add_to(m)

    # Salva o mapa em um arquivo HTML
    m.save("mapa_equipamentos_publicos.html")
    print("\nMapa 'mapa_equipamentos_publicos.html' foi gerado com sucesso! Abra este arquivo em seu navegador.")

Feição para 'Biblioteca Mário de Andrade, São Paulo, Brasil' encontrada com sucesso!
Feição para 'Campo de Marte, São Paulo, Brasil' encontrada com sucesso!
Feição para 'Centro Cultural da Penha, São Paulo, Brasil' encontrada com sucesso!
Não foi possível encontrar uma feição para 'Centro Cultural Tendal da Lapa, São Paulo, Brasil': Nominatim geocoder returned 0 results for query 'Centro Cultural Tendal da Lapa, São Paulo, Brasil'.
Não foi possível encontrar uma feição para 'Centro Cultural Vila Itororó, São Paulo, Brasil': Nominatim did not geocode query 'Centro Cultural Vila Itororó, São Paulo, Brasil' to a geometry of type (Multi)Polygon.
Feição para 'Centro TEA, São Paulo, Brasil' encontrada com sucesso!
Não foi possível encontrar uma feição para 'CEU Arthur Alvim - Abdias do Nascimento, São Paulo, Brasil': Nominatim geocoder returned 0 results for query 'CEU Arthur Alvim - Abdias do Nascimento, São Paulo, Brasil'.
Feição para 'CEU Carrão, São Paulo, Brasil' encontrada com sucesso!

C:\Users\vitorio.aflalo\AppData\Local\Temp\ipykernel_2788\3254417688.py:77: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  lat_media = todos_os_poligonos.unary_union.centroid.y
C:\Users\vitorio.aflalo\AppData\Local\Temp\ipykernel_2788\3254417688.py:78: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  lon_media = todos_os_poligonos.unary_union.centroid.x



Mapa 'mapa_equipamentos_publicos.html' foi gerado com sucesso! Abra este arquivo em seu navegador.
